In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

spark = SparkSession.builder \
    .appName("InfoTransportes") \
    .getOrCreate()

In [2]:
df_bronze = spark.read.csv("info_transportes.csv",
                    sep = ';',
                    header = True,
                    inferSchema = True,
)
df_bronze.show()
df_bronze.printSchema()

+----------------+----------------+---------+-----------------+---------------+---------+-----------------+
|     DATA_INICIO|        DATA_FIM|CATEGORIA|     LOCAL_INICIO|      LOCAL_FIM|DISTANCIA|        PROPOSITO|
+----------------+----------------+---------+-----------------+---------------+---------+-----------------+
|01-01-2016 21:11|01-01-2016 21:17|  Negocio|      Fort Pierce|    Fort Pierce|       51|      Alimentação|
|01-02-2016 01:25|01-02-2016 01:37|  Negocio|      Fort Pierce|    Fort Pierce|        5|             null|
|01-02-2016 20:25|01-02-2016 20:38|  Negocio|      Fort Pierce|    Fort Pierce|       48|         Entregas|
|01-05-2016 17:31|01-05-2016 17:45|  Negocio|      Fort Pierce|    Fort Pierce|       47|          Reunião|
|01-06-2016 14:42|01-06-2016 15:49|  Negocio|      Fort Pierce|West Palm Beach|      637|Visita ao cliente|
|01-06-2016 17:15|01-06-2016 17:19|  Negocio|  West Palm Beach|West Palm Beach|       43|      Alimentação|
|01-06-2016 17:30|01-06-2016

In [3]:
df_silver = df_bronze.withColumn(
    'DT_REFE', f.date_format(f.to_timestamp(f.col('DATA_INICIO'), 'MM-dd-yyyy H:mm'), 'yyyy-MM-dd')) \
        .withColumn('DISTANCIA', f.when((f.col('DISTANCIA') <= 0) | f.col('DISTANCIA').isNull(), 'Nao-consta').otherwise(f.col('DISTANCIA'))) \
        .fillna(value='Nao-consta', subset=['CATEGORIA', 'LOCAL_INICIO', 'LOCAL_FIM', 'PROPOSITO']) \

df_silver.show()

+----------------+----------------+---------+-----------------+---------------+---------+-----------------+----------+
|     DATA_INICIO|        DATA_FIM|CATEGORIA|     LOCAL_INICIO|      LOCAL_FIM|DISTANCIA|        PROPOSITO|   DT_REFE|
+----------------+----------------+---------+-----------------+---------------+---------+-----------------+----------+
|01-01-2016 21:11|01-01-2016 21:17|  Negocio|      Fort Pierce|    Fort Pierce|       51|      Alimentação|2016-01-01|
|01-02-2016 01:25|01-02-2016 01:37|  Negocio|      Fort Pierce|    Fort Pierce|        5|       Nao-consta|2016-01-02|
|01-02-2016 20:25|01-02-2016 20:38|  Negocio|      Fort Pierce|    Fort Pierce|       48|         Entregas|2016-01-02|
|01-05-2016 17:31|01-05-2016 17:45|  Negocio|      Fort Pierce|    Fort Pierce|       47|          Reunião|2016-01-05|
|01-06-2016 14:42|01-06-2016 15:49|  Negocio|      Fort Pierce|West Palm Beach|      637|Visita ao cliente|2016-01-06|
|01-06-2016 17:15|01-06-2016 17:19|  Negocio|  W

In [4]:
info_corridas_do_dia = df_silver.groupBy('DT_REFE').agg(
    f.count('*').alias('QT_CORR'),
    f.count(f.when(f.col('CATEGORIA') == 'Negocio', True)).alias('QT_CORR_NEG'),
    f.count(f.when(f.col('CATEGORIA') == 'Pessoal', True)).alias('QT_CORR_PESS'),  
    f.round(f.max('DISTANCIA'),1).alias('VL_MAX_DIST'),
    f.round(f.min('DISTANCIA'),1).alias('VL_MIN_DIST'),
    f.round(f.avg('DISTANCIA').alias('VL_AVG_DIST'), 1).alias('VL_AVG_DIST'),
    f.count(f.when(f.col('PROPOSITO') == 'Reunião', True)).alias('QT_CORR_REUNI'),
    f.count(f.when(f.col('PROPOSITO') != 'Reunião', True)).alias('QT_CORR_NAO_REUNI'),
)

#info_corridas_do_dia = info_corridas_do_dia.orderBy('DT_REFE')

info_corridas_do_dia.show()

+----------+-------+-----------+------------+-----------+-----------+-----------+-------------+-----------------+
|   DT_REFE|QT_CORR|QT_CORR_NEG|QT_CORR_PESS|VL_MAX_DIST|VL_MIN_DIST|VL_AVG_DIST|QT_CORR_REUNI|QT_CORR_NAO_REUNI|
+----------+-------+-----------+------------+-----------+-----------+-----------+-------------+-----------------+
|2016-01-01|      1|          1|           0|       51.0|       51.0|       51.0|            0|                1|
|2016-01-02|      2|          2|           0|        5.0|       48.0|       26.5|            0|                2|
|2016-01-05|      1|          1|           0|       47.0|       47.0|       47.0|            1|                0|
|2016-01-06|      3|          3|           0|       71.0|       43.0|      250.3|            1|                2|
|2016-01-07|      1|          1|           0|        8.0|        8.0|        8.0|            1|                0|
|2016-01-10|      5|          5|           0|       83.0|      108.0|       98.6|       